#### 사용자 정의 데이터셋과 모델, 학습
- iris.csv => 사용자 정의 데이터셋
- DNN 모델 => 사용자 정의 모델

[1] 모듈 로딩 및 데이터 준비 <hr>

In [7]:
# 모듈로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opimizer
from torchmetrics.classification import F1Score
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder # 타겟 컬럼 수치화

In [8]:
# 데이터
DATA_FILE= '../data/iris.csv'

In [9]:
# CSV => DataFrame
irisDF=pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [10]:
# 타겟 컬럼 수치화 => LabelEncoder
encoder=LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety']=encoder.transform(irisDF['variety'])

[2] 사용자 정의 데이터셋 클래스 생성

In [11]:
# -------------------------------------------------------------------------
# 클래스목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스이름 : customDataSet
# 부모클래스 : torch.utils.data.Dataset
# 매개변수 : featureDF, targetDF
# -------------------------------------------------------------------------

In [12]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

In [13]:
#------------------------------------------------------------------------------------
# 함수기능 : 파일 확장자별 데이터프레임 변환 기능
# 함수이름 : convertDataFrame
# 매개변수 : data_path
# 함수결과 : DataFrame
#------------------------------------------------------------------------------------
def convertDataFrame(data_path, exit_header=0):
    ext=data_path.rsplit('.')[-1]
    if ext=='csv':
        return pd.read_csv(data_path, header=exit_header)
    elif ext=='json':
        return pd.read_json(data_path, header=exit_header)
    elif ext in ['xlsx','xls']:
        return pd.read_excel(data_path, header=exit_header)
    else : return pd.read_table(data_path, header=exit_header)

In [14]:
#------------------------------------------------------------------------------------
# 클래스기능 : 파일기반 데이터셋
# 클래스이름 : FileDataset
# 매개변수 : data_path 파일 경로
# 부모클래스 : utils.data.Dataset
#------------------------------------------------------------------------------------
class FileDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, data_path):
        super().__init__()
        # 데이터파일 => DataFrame으로 변환
        dataDF=convertDataFrame(data_path)
        # DataFrame => 피쳐와 타겟 추출
        self.featuresDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[-1:]]

        self.n_features=self.dataDF.shape[1]
        self.n_rows=self.featuresDF.shape[0]
    
    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성 <hr>

In [15]:
featureDF, targetDF=irisDF[irisDF.columns[:-1]], irisDF[irisDF.columns[-1:]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [16]:
# IRIS 데이터셋 인스턴스 생성
irisDS=CustomDataset(featureDF, targetDF)

In [17]:
# IRIS 데이터셋 속성
irisDS.n_features, irisDS.n_rows

(4, 150)

In [18]:
# IRIS 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [19]:
# 필요한 것 : Dataset 인스턴스, Batch_size
irisDL=DataLoader(irisDS, batch_size=3)

In [20]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break

torch.Size([3, 4]) torch.Size([3, 1])


[5] 모델 준비<hr>

In [21]:
# ---------------------------------------------------------------------------------
# 모델이름 : CustomModel
# 부모클래스 : nn.Module
# 매개변수 : None
# 모델구조
# - 입력층 : 입력  4개 출력 10개    AF ReLU => LeakyReLU
# - 은닉층 : 입력 10개 출력 30개    AF ReLU => LeakyReLU
# - 출력층 : 입력 30개 출력  3개    AF 분류 - 다중 softmax
# ---------------------------------------------------------------------------------
class CustomModel(nn.Module):
    # 모델 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.hidden_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,3)

    # 순방향 학습 메서드
    def forward(self, x):
        y=F.relu(self.in_layer(x))
        y=F.relu(self.hidden_layer(y))
        return self.out_layer(y)


In [22]:
# 모델 인스턴스 생성
model=CustomModel()
print(model)
summary(model, input_size=(1000, 4))

CustomModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
CustomModel                              [1000, 3]                 --
├─Linear: 1-1                            [1000, 10]                50
├─Linear: 1-2                            [1000, 30]                330
├─Linear: 1-3                            [1000, 3]                 93
Total params: 473
Trainable params: 473
Non-trainable params: 0
Total mult-adds (M): 0.47
Input size (MB): 0.02
Forward/backward pass size (MB): 0.34
Params size (MB): 0.00
Estimated Total Size (MB): 0.36

[6] 학습<hr>

In [24]:
# 배치크기만큼 데이터와 타겟 추출해서 학습 진행
TS_loss, TS_score=[[],[]], [[],[]]
for epoch in range(10):
    loss_total, score_total=0, 0
    for dataTS, targetTS in irisDL:

        # 배치크기 만큼의 학습 데이터
        print(dataTS.shape, targetTS.shape)
        targetTS=targetTS.reshape(-1).long()

        # 배치크기만큼 학습 진행
        pre_y=model(dataTS)

        # 손실 계산
        loss=nn.CrossEntropyLoss()(pre_y, targetTS.reshape(-1))

        break

torch.Size([3, 4]) torch.Size([3, 1])
